In [34]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from google.colab import drive
import PIL
import os
import numpy as np
import pandas as pd
import torch
import random
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data_path = '/content/drive/MyDrive/Colab Notebooks/age_detection/data/data/'

In [7]:
counts = {}
def count_files(start_path):
  counts[start_path] = 0
  for name in os.listdir(start_path):
    if os.path.isdir(f'{start_path}{name}/'):
      count_files(f'{start_path}{name}/')
    else:
      counts[start_path] += 1

count_files(data_path)

In [18]:
del transforms

In [8]:
counts

{'/content/drive/MyDrive/Colab Notebooks/age_detection/data/data/': 0,
 '/content/drive/MyDrive/Colab Notebooks/age_detection/data/data/0-2/': 1605,
 '/content/drive/MyDrive/Colab Notebooks/age_detection/data/data/13-18/': 1082,
 '/content/drive/MyDrive/Colab Notebooks/age_detection/data/data/19-24/': 2408,
 '/content/drive/MyDrive/Colab Notebooks/age_detection/data/data/25-35/': 8204,
 '/content/drive/MyDrive/Colab Notebooks/age_detection/data/data/3-12/': 1808,
 '/content/drive/MyDrive/Colab Notebooks/age_detection/data/data/36-44/': 2548,
 '/content/drive/MyDrive/Colab Notebooks/age_detection/data/data/45-60/': 3656,
 '/content/drive/MyDrive/Colab Notebooks/age_detection/data/data/60+/': 2397}

In [22]:
dataset_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(45),
    transforms.RandomAutocontrast(),
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


dataset = datasets.ImageFolder(data_path, transform=dataset_transforms)

* Link PDF: https://drive.google.com/drive/u/0/folders/1MZ9lhzGKABXLMTJzeIM2b-zi7YJnn6WR
* Link Chess: https://github.com/LeventSoykan/Chess_Piece_Image_Classification_With_CNN/blob/main/image_classification.ipynb
* Dataset: https://susanqq.github.io/UTKFace/


In [23]:
#GPU Batch Size of 32 used, and random seeds prepared for consistency among splitting & training processes
validation_split = .2
test_split = .3
batch_size = 32
seed_val = 1903
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [24]:
#Random shuffling indices before split to get similar distribution among classes
dataset_size = len(dataset)
indices = list(range(dataset_size))
np.random.shuffle(indices)

In [25]:
split_test = int(np.floor(test_split * dataset_size))
split_val = int(np.floor(validation_split * (dataset_size-split_test)))+split_test
split_test, split_val, dataset_size

(7112, 10431, 23708)

In [26]:
#Create Train-Test-Validation Datasets
train_indices, val_indices, test_indices = indices[split_val:], indices[split_test:split_val], indices[:split_test]

In [27]:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, drop_last=True)
val_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=test_sampler, drop_last=True)

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(45),
    transforms.RandomAutocontrast(),
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset_train = datasets.ImageFolder(
    data_path_train,
    transform = train_transforms
)

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset_test = datasets.ImageFolder(
    data_path_test,
    transform = test_transforms
)



In [ ]:
dataloader_train = DataLoader(
    dataset_train,
    shuffle = True,
    batch_size = 1
)

image, label = next(iter(dataloader_train))
print(image.shape)

torch.Size([1, 3, 128, 128])


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
image = image.squeeze().permute(1, 2, 0)
print(image.shape)

torch.Size([128, 128, 3])


In [30]:
dataset.shape

AttributeError: 'ImageFolder' object has no attribute 'shape'

In [31]:
class AgeDetectNet(nn.Module):
  def __init__(self, num_classes):
    super().__init__()
    self.feature_extractor = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Flatten()
    )
    self.classifier = nn.Linear(16*32*32, num_classes)

  def forward(self, x):
    x = self.feature_extractor(x)
    x = self.classifier(x)
    return x

In [38]:
net = AgeDetectNet(num_classes=8)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
print(net)

AgeDetectNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Flatten(start_dim=1, end_dim=-1)
  )
  (classifier): Linear(in_features=16384, out_features=8, bias=True)
)


In [39]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# number of epochs to train the model
n_epochs = 2

device = 'cpu'
model.to(device)

t0 = time.time()


valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    train_accuracy = 0.0
    valid_accuracy = 0.0

    ###################
    # train the model #
    ###################
    model.train()
    i=0
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(images)
        # calculate batch accuracy
        probs = nn.functional.softmax(output, dim=1)
        preds = torch.argmax(probs, axis=1)
        accuracy = (abs(preds-labels)==0).type(torch.float).mean()
        # calculate the batch loss
        loss = criterion(output, labels)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss & accuracy
        train_loss += loss.item()*images.size(0)
        train_accuracy += accuracy*images.size(0)

        #inform user about elapsed time in every 320 batches (320x32=10240 images)
        i += 1
        if i % 320 == 0:
            t1 = time.time()
            print(f'{i*images.shape[0]} items processed in {format_time(t1-t0)}')


    ######################
    # Validate the model #
    ######################
    model.eval()
    for images, labels in val_dataloader:
        images, labels = images.to(device), labels.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(images)
        # calculate batch accuracy
        probs = nn.functional.softmax(output, dim=1)
        preds = torch.argmax(probs, axis=1)
        accuracy = (abs(preds-labels)==0).type(torch.float).mean()
        # calculate the batch loss
        loss = criterion(output, labels)
        # update average validation loss & accuracy
        valid_loss += loss.item()*images.size(0)
        valid_accuracy += accuracy*images.size(0)

    # calculate average losses & accuracies
    train_loss = train_loss/len(train_dataloader.sampler)
    valid_loss = valid_loss/len(val_dataloader.sampler)
    train_accuracy = train_accuracy/len(train_dataloader.sampler)
    valid_accuracy = valid_accuracy/len(val_dataloader.sampler)

    # print training/testing statistics
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    print('Epoch: {} \tTraining Accuracy: {:.6f} \tValidation Accuracy: {:.6f}'.format(
        epoch, train_accuracy, valid_accuracy))

    # save model if testing loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_chess.pt')
        valid_loss_min = valid_loss

In [ ]:
net = Net(num_classes=5)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(10):
  for images,labels in dataloader_train:
    optimizer.zero_grad()
    outputs = net(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()